In [1]:
from pprint import pprint
from time import time
import pathlib
import pandas as pd
import eli5
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

In [2]:
# Load pre-processed train and test data
data_path = pathlib.Path().cwd().parent / "data"
print(f"Logging info - Loading data from {data_path}")
train_df = pd.read_csv(f"{data_path}/train.csv", keep_default_na=False)
test_df = pd.read_csv(f"{data_path}/test.csv", keep_default_na=False)

Logging info - Loading data from /Users/1150704/Documents/mywork/zip_product_classification/data


In [3]:
# build a pipeline 
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,3), stop_words='english', 
                                              sublinear_tf=True, max_features=50000, min_df=2)),
                     ('clf', LogisticRegression(n_jobs=4, solver='lbfgs', 
                                                random_state=17, verbose=0, 
                                                multi_class='multinomial',
                                                penalty="l2",
                                                fit_intercept=True))
                    ])

Pipeline(steps=[('vect',
                 TfidfVectorizer(max_features=50000, min_df=2,
                                 ngram_range=(1, 3), stop_words='english',
                                 sublinear_tf=True)),
                ('clf',
                 LogisticRegression(multi_class='multinomial', n_jobs=4,
                                    random_state=17))])

## Hyper-parameter tuning

In [6]:
# Tune the weight that controls the strength of penalty (Smaller values specify stronger regularization)
parameters = {
    "clf__C": [1, 10, 100] 
}

In [7]:
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, cv=5, scoring="accuracy")

print("Logging info - Performing grid search")
print("parameters:")
pprint(parameters)
t0 = time()
grid_search.fit(train_df["desc"], train_df["cat0_cat1_cat2"])
print("done in %0.3fs" % (time() - t0))

Logging info - Performing grid search
parameters:
{'clf__C': [1, 10, 100]}
Fitting 5 folds for each of 3 candidates, totalling 15 fits


/Users/1150704/venvs/zip_product_classification/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/1150704/venvs/zip_product_classification/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.

done in 298.392s


/Users/1150704/venvs/zip_product_classification/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
print(grid_search.best_score_)
print(grid_search.best_params_)

In [ ]:
# Define pipeline with best parameter and fit on training data
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,3), stop_words='english', 
                                              sublinear_tf=True, max_features=50000, min_df=2)),
                     ('clf', LogisticRegression(C=100, n_jobs=4, solver='lbfgs', 
                                                random_state=17, verbose=0, 
                                                multi_class='multinomial',
                                                penalty="l2",
                                                fit_intercept=True))
                    ])
pipeline.fit(train_df['desc'], train_df['cat0_cat1_cat2'])

In [ ]:
pred_level_3 = pipeline.predict(test_df['desc'])

In [ ]:
pred_level_0 = [el.split('|')[0] for el in pred_level_3]
pred_level_1 = [el.split('|')[1] for el in pred_level_3]
pred_level_2 = [el.split('|')[2] for el in pred_level_3]

## Evaluation

In [ ]:
print(classification_report(
    y_true=test_df['cat0'], 
    y_pred=pred_level_0,
    digits=4)
)

In [ ]:
print(classification_report(
    y_true=test_df['cat1'], 
    y_pred=pred_level_1,
    digits=4)
)

In [ ]:
print(classification_report(
    y_true=test_df['cat2'], 
    y_pred=pred_level_2,
    digits=4)
)

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title='Confusion matrix', figsize=(7,7),
                          cmap=plt.cm.Blues, path_to_save_fig=None):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    cm = confusion_matrix(y_true, y_pred).T
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    plt.figure(figsize=figsize)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Predicted label')
    plt.xlabel('True label')
    
    if path_to_save_fig:
        plt.savefig(path_to_save_fig, dpi=300, bbox_inches='tight')

In [ ]:
plot_confusion_matrix(
    y_true=test_df['cat0'],
    y_pred=pred_level_0, 
    classes=sorted(test_df['cat0'].unique()),
    figsize=(8, 8)
)

In [ ]:
plot_confusion_matrix(
    y_true=test_df['cat1'],
    y_pred=pred_level_1, 
    classes=sorted(test_df['cat1'].unique()),
    figsize=(8, 8)
)

In [ ]:
plot_confusion_matrix(
    y_true=test_df['cat2'],
    y_pred=pred_level_2, 
    classes=sorted(test_df['cat2'].unique()),
    figsize=(10, 10)
)

## Explainations

In [ ]:
eli5.show_weights(
    estimator=pipeline.named_steps['clf'],
    vec=pipeline.named_steps['vect'])

In [ ]:
# Save model
import joblib

joblib.dump(pipeline, 'baseline_model.joblib')